## Qualtrics to Filemaker pipeline

This notebook takes two input files:
1. The survey output csv file downloaded from Qualtrics.
2. An excel file that organizes words into their Parts of Speech (POS).

<br>This notebook exports one file:
1. A csv file with summary calculations that can be imported into Filemaker.

#### <br><br>Input file 1: Qualtrics survey csv
To export data from Qualtrics:
- Select the correct survey.
- Click on Data & Analysis on the top menu
- Click on the Export & Import button (near the top right of the page)
- Choose Export Data...
- Choose CSV on the top menu of the pop up
- Make sure "Download all fields" is checked
- Make sure "Use choice text" is selected
- Click Download
<br><br>Other instructions:
- You must tell this notebook the name of the qualtrics file you are using below, where it says "define input file names". You would change the file name being assigned to the `file` variable name. The file you are using must be in the same folder as this notebook, or you must include the relative or absolute path to the file (i.e. instead of "Sample Survey.csv" you would say "~/Documents/mySurvey/Sample Survey.csv").

#### <br><br>Input file 2: POS excel file

- Excel (xls or xlsx) file
- Only one sheet in the excel file
- Parts of Speech are listed in the first row (column names), ex. noun, verb, adjective
- Words to be counted as each POS are listed below each appropriate column header
- Go to https://github.com/aGitHasNoName/waxmanConsult/blob/main/POSmain.xlsx and click on View raw to download a sample file.
- It is ok if the POS file has more words than are used in your survey. (If you want, you can have one main file where you assign words to POS for all surveys, or you can have separate files for different surveys/researchers/etc.)
- It is ok if the survey uses more words than are in the pos file. Words that are not assigned to a POS in this file will be counted in the total word counts, but will not be included in the POS word counts.
- You must tell this notebook the name of the POS file you are using below, where it says "define input file names". You would change the file name being assigned to the `pos_file` variable name. The POS file you are using must be in the same folder as this notebook, or you must include the relative or absolute path to the POS file (i.e. instead of "POSmain.xlsx" you would say "~/Documents/mySurvey/POSmain.xlsx").

#### <br><br>To run the code in this notebook
- First, change the input file names as explained above.
- Go to Run in the menu above, choose Run All Cells.
- If you get an error when defining file names, recheck that the file names match exactly and that they are in the same folder as this notebook (or you have used relative or absolute paths). You can click on the File Browser icon on the left side menu to view all the files in this folder.
- If you need to rerun the code after getting an error, you can go to Run in the menu above and choose Restart Kernel and Run All Cells.

In [1]:
# import required packages
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from tkinter.simpledialog import askstring

In [2]:
# user must supply survey name to be matched in the Filemaker database
root = Tk()
root.withdraw()
survey_name = askstring("Survey Name", "Enter the name of this survey for the database:")
root.destroy()

In [3]:
# let user select input file in browser
"""Choose the Qualtrics Survey csv 
file you would like to convert."""
root = Tk()
root.withdraw()
file = askopenfilename(filetypes=[(".csv file", ".csv")],title="Choose the Qualtrics Survey csv file you would like to convert")
root.destroy()

In [4]:
# let user select parts of speech file in browser
"""Choose the Parts of Speech excel 
file you would like to use as a reference."""
root = Tk()
root.withdraw()
pos_file = askopenfilename(filetypes=[("excel file", ".xlsx"), ("old excel file", ".xls")],title="Choose the Parts of Speech excel file you would like to use as a reference")
root.destroy()

In [5]:
# collect parts of speech
pos_dict = {k:list(v.dropna()) for k,v in pd.read_excel(pos_file).to_dict("series").items()}

In [6]:
# load survey input data
df = pd.read_csv(file, skiprows=1)

In [7]:
# translate long qualtrics column headers to short column headers
header_dict = {"Recorded Date": "RecordedDate",
               "Please provide your email address (to receive your payment): - Email": "email", 
               "Child's First and Last Name(we need this to ensure that survey responses are matched correctly with the experimental data; after that, the data will be anonymized):": "child_name", 
               "Child's Birthdate (example: December 15, 2012)": "birthday", 
               "Which language(s), other than English, is/are spoken at home? If English is the only language spoken, please write n/a.": "language", 
               "What percentage of the time does your child hear this language?": "lang_percent"}
vocab_dict = {i:i[i.rfind("-")+2:] for i in [i for i in df.columns if "Vocabulary checklist" in i]}
df = df.rename(header_dict, axis=1)
df = df.rename(vocab_dict, axis=1)

In [8]:
# remove other columns that aren't being used
df = df[list(header_dict.values()) + list(vocab_dict.values())]

In [9]:
# drop second header row
df = df.drop(0)

In [10]:
# convert dataframe to dictionary
df_d = df.to_dict(orient="records")

In [11]:
# remove time and keep only date survey was taken
for i in df_d:
    i["RecordedDate"] = i["RecordedDate"].split(" ")[0]

In [12]:
# collect and count words by total, understands, says, and all parts of speech
for i in df_d:
    u_list = [k.replace(" ", "-") for k,v in i.items() if v=="Understands"]
    s_list = [k.replace(" ", "-") for k,v in i.items() if v=="Understands and says"]
    t_list = u_list + s_list
    pos_lists = {k: [word for word in t_list if word.replace("-", " ") in v] for k,v in pos_dict.items()}
    # add words and counts to dictionary
    i["UnderstandsWords"] = (" ").join(u_list)
    i["UnderstandsCount"] = len(u_list)
    i["SaysWords"] = (" ").join(s_list)
    i["SaysCount"] = len(s_list)
    i["TotalWords"] = (" ").join(t_list)
    i["TotalCount"] = len(t_list)
    for k, v in pos_lists.items():
        i[k + "Words"] = (" ").join(v)
        i[k + "Count"] = len(v)

In [13]:
# remove individual word columns
keep_keys = ["RecordedDate", "email", "child_name", 
             "birthday", "language", "lang_percent", 
             "UnderstandsWords", "UnderstandsCount", 
             "SaysWords", "SaysCount",
             "TotalWords", "TotalCount"]
keep_keys = keep_keys + [i + "Words" for i in pos_lists.keys()] + [i + "Count" for i in pos_lists.keys()]
final_dict = [{k:v for k,v in i.items() if k in keep_keys} for i in df_d]

In [14]:
# write final data to output file
final_df = pd.DataFrame.from_dict(final_dict)
final_df.to_csv(file.replace(".csv", "OUT.csv"), index=False)